<a href="https://colab.research.google.com/github/Alixblk/TraitementDonn-es/blob/main/notebooks/Adapters_Language_Transfer_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tanikina/low-resource-nlp-lab/blob/main/notebooks/Adapters_Language_Transfer_Exercise.ipynb)

In [1]:
!pip install transformers[torch]
!pip install adapters
!pip install datasets
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 10.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.3
    Uninstalling transformers-4.46.3:
      Successfully uninstalled transformers-4.46.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's depe

### 🗃️ Dataset Preparation
**The Choice Of Plausible Alternatives** (COPA) is a dataset for commonsense causal reasoning. It consists of questions where each question has a premise and two alternatives and the task is to select the correct alternative given the premise.

In [2]:
from datasets import load_dataset
# Loading the COPA dataset
dataset_en = load_dataset("super_glue", "copa")
dataset_en.num_rows

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/18.2k [00:00<?, ?B/s]

super_glue.py:   0%|          | 0.00/30.7k [00:00<?, ?B/s]

The repository for super_glue contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/super_glue.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

{'train': 400, 'validation': 100, 'test': 500}

In [3]:
dataset_en['train'][42]

{'premise': 'The executive decided not to hire the applicant.',
 'choice1': 'The applicant failed a background check.',
 'choice2': 'The applicant had experience for the job.',
 'question': 'cause',
 'idx': 42,
 'label': 0}

In [4]:
class DatasetPreprocessor:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    """Encodes a batch of input data."""
    def encode_batch(self, examples):
      all_encoded = {"input_ids": [], "attention_mask": []}
      for premise, question, choice1, choice2 in zip(examples["premise"], examples["question"], examples["choice1"], examples["choice2"]):
        sentences_a = [premise + " " + question for _ in range(2)]
        # Both answer choices are passed in an array according to the format needed for the multiple-choice prediction head
        sentences_b = [choice1, choice2]
        encoded = self.tokenizer(
            sentences_a,
            sentences_b,
            max_length=60,
            truncation=True,
            padding="max_length",
        )
        all_encoded["input_ids"].append(encoded["input_ids"])
        all_encoded["attention_mask"].append(encoded["attention_mask"])
      return all_encoded

    """Encodes input data"""
    def preprocess_dataset(self, dataset):
      dataset = dataset.map(self.encode_batch, batched=True)
      # The transformers model expects the target class column to be named "labels".
      dataset = dataset.rename_column("label", "labels")
      # Transform to pytorch tensors and only output the required columns.
      dataset.set_format(columns=["input_ids", "attention_mask", "labels"])
      return dataset

# Defining the tokenizer and pre-processing the data
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
dp = DatasetPreprocessor(tokenizer)
dataset_en = dp.preprocess_dataset(dataset_en)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

### ⁉️ Task 1
1. **Load** the `xlm-roberta-base` model.
2. **Find** language adapters for English and Swahili on the AdapterHub: [https://adapterhub.ml/explore/text_lang/](https://adapterhub.ml/explore/text_lang/).

   Don't forget to make sure when selecting the adapter that it is compatible with the base model. Then load the selected adapters into the base model.
4. **Add** the task adapter and the multiple choice head. You can refer to the following documentation for this: [https://docs.adapterhub.ml/](https://docs.adapterhub.ml/).
5. **Set** the task adapter in the training mode and stack the language adapters (first language, then task adapter).

In [18]:
from adapters.composition import Stack
from transformers import AutoConfig
from adapters import AutoAdapterModel, AdapterConfig
from datasets import DatasetDict

In [22]:
# TODO: follow the instructions for Task 1
# Questions 1 et 2
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
dp = DatasetPreprocessor(tokenizer)

# Vérification du type de dataset et renommer la colonne
if isinstance(dataset_en, DatasetDict):
    # Appliquer rename_column à chaque split
    for split_name in dataset_en:
        dataset_en[split_name] = dataset_en[split_name].rename_column("labels", "label")
else:
    # Si ce n'est pas un DatasetDict, appliquer directement
    dataset_en = dataset_en.rename_column("labels", "label")

dataset_en = dp.preprocess_dataset(dataset_en)

model = AutoAdapterModel.from_pretrained("xlm-roberta-base")

config = AdapterConfig.load("pfeiffer", non_linearity = "relu", reduction_factor = 2)
model.load_adapter("sw/wiki@ukp", config = config)
model.load_adapter("en/wiki@ukp", config = config)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/adapters/loading.py:165: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_file, map_location="cpu")


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

'en'

In [26]:
# Question 3

# Charger le modèle de base
model = AutoAdapterModel.from_pretrained("xlm-roberta-base")

# Ajouter un task adapter pour une tâche multiple choice
task_adapter_config = AdapterConfig.load("pfeiffer", non_linearity="relu", reduction_factor=2)
model.add_adapter("multiple_choice_task", config=task_adapter_config)

# Ajouter une "multiple choice head"
model.add_prediction_head("multiple_choice_head", head_type="multiple_choice", num_choices=4)

# Activer le mode entraînement pour l'adapter
model.train_adapter("multiple_choice_task")

# Empiler les adapters (langues d'abord, puis tâche)
model.set_active_adapters([["en/wiki@ukp", "sw/wiki@ukp"], "multiple_choice_task"])

Some weights of XLMRobertaAdapterModel were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: ModelWithFlexibleHeadsAdaptersMixin.add_prediction_head() got an unexpected keyword argument 'head_type'

### 🚀 Training

In [20]:
from transformers import TrainingArguments
from adapters import AdapterTrainer
from datasets import concatenate_datasets

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=100,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

train_dataset = concatenate_datasets([dataset_en["train"], dataset_en["validation"]])

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

ValueError: Expected a model with an active adapter setup.If you want to fully finetune the model use the Trainer class.

### ✅ Evaluation
**XCOPA** is a multilingual dataset for causal commonsense reasoning (similarly to COPA) in 11 languages. It includes some resource-poor languages like Eastern Apurímac Quechua and Haitian Creole. In this example we will work with Swahili.

In [ ]:
# Loading the XCOPA dataset for Swahili
dataset_sw = load_dataset("xcopa", "sw", verification_mode=None)
print(dataset_sw["test"][5])
dataset_sw = dp.preprocess_dataset(dataset_sw)

{'premise': 'Msichana alipungua nguvu.', 'choice1': 'Alicheza drafti.', 'choice2': 'Aliruka kamba.', 'question': 'cause', 'label': 1, 'idx': 5, 'changed': False}


### ⁉️ Task 2
1. **Swap** the language adapter for English with the adapter for Sawhili. Evaluate the model using the stacked adapters (execute the cell below).
2. **Remove** the language adapter and evauate the results using only the task adapter.
3. (Optional) Experiment with a **different language** (e.g., XCOPA supports Turkish, Tamil, Mandarin Chinese etc., you can find pre-trained adapters on [https://adapterhub.ml/explore](https://adapterhub.ml/explore))
4. (Optional) Experiment with a **different model** (e.g., replace `xlm-roberta-base` with `bert-base-multilingual-cased`) and play around with different hyperparameters (learning rate, number of epochs etc.)

In [ ]:
# TODO: follow the instructions for Task 2

In [ ]:
import numpy as np
from transformers import EvalPrediction

print("Active adapters:", model.active_adapters)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}
eval_trainer = AdapterTrainer(
    model=model,
    args=TrainingArguments(output_dir="./eval_output", remove_unused_columns=False,),
    eval_dataset=dataset_sw["test"],
    compute_metrics=compute_accuracy,
)
eval_trainer.evaluate()

⁉️ [Continue here to check the solution!](https://github.com/tanikina/low-resource-nlp-lab/blob/main/notebooks/Adapters_Language_Transfer_Exercise_Solution.ipynb)